The problem that we aim to solve is a sampled average optimization problem that has the following form:
\begin{align}\label{SAA: dual_oce_b1}
\min_{\substack{\lambda\geq 0\\\theta_1,\theta_2\in \mathbb
    R}}-\theta_1-\theta_2+\lambda r +\frac{1}{N}\sum^N_{i=1}\lambda \phi^*\left(\frac{\max\{\frac{1}{\alpha}(\theta_2-X_i),0\}+\theta_1}{\lambda}\right),
\end{align}
where $r,\alpha>0$ are parameters constants (for example: $r=0.01, \alpha=0.05$) and  $\phi^*$ is the following convex function, twice differentiable, strictly increasing and defined over the entire real line:
 \begin{align*}
        \phi^*(x)=\begin{cases}
           c_1(x+e)e^{0.1\cdot\log^2(x+e)}+c_2x+c_3& x>0\\
            e^x-1& x\leq 0,
        \end{cases}
    \end{align*}
where $c_1= 5.59, c_2=-6.413, c_3 = -16.793$ are predetermined constants, chosen such that the first and second derivative of $\phi^*$ at $x=0$ are equal from left and right, to ensure twice differentiability of $\phi^*$ also at $x=0$.

It is not really clear if this $\phi^*$ is conic-representable, due to the presence of the $\log^2(x+e)$ term in the exponent which is not convex for $x>0$. However, one can check by second derivative, that $\phi^*$ is strictly convex. 

In this problem, $X_i$'s are the i.i.d. samples drawn from a log-normal distribution (with mean zero, variance one). $N$ denotes the number of samples.

Since the maximum function is non-differentiable, we introduce extra variables to rewrite the above optimization into an optimization problem with differentiable objective functions and constraint functions:

\begin{align}
\min_{\substack{\lambda\geq 0\\\theta_1,\theta_2\in \mathbb
    R\\ s_1,\ldots, s_N\geq 0}}&~-\theta_1-\theta_2+\lambda r +\frac{1}{N}\sum^N_{i=1}\lambda \phi^*\left(\frac{s_i+\theta_1}{\lambda}\right)\\
    \text{subject to}&~\frac{1}{0.05}(\theta_2-X_i)\leq s_i,~i=1,\ldots,N.
\end{align}

Since the objective functions and the constraint functions here are all twice differentiable, our idea is to use ipopt to solve this optimization problem. 

We import the necessary packages for running the ipopt solver:

In [2]:
import numpy as np
import cvxpy as cp
import cyipopt    #### the python ipopt wrapper

To apply ipopt, we need to calculate the gradient and the hessian of the objective and constraint function. To do this, we first start calculating the derivative of our $\phi^*$ function, which is given by:
\begin{align*}
        (\phi^*)'(x)=\begin{cases}
           c_1e^{0.1\cdot\log^2(x+e)}(1+0.2\log(x+e))+c_2& x>0\\
            e^x& x\leq 0.
        \end{cases}
    \end{align*}
The second derivative $(\phi^*)''$ is given by:
\begin{align*}
        (\phi^*)''(x)=\begin{cases}
           \frac{0.2c_1}{x+e}e^{0.1\cdot\log^2(x+e)}(0.2\log^2(x+e)+\log(x+e)+1)& x>0\\
            e^x& x\leq 0.
        \end{cases}
    \end{align*}



We implement the function value, derivative and second derivative of $\phi^*$ below:

In [13]:
def f_explog(x,a,b):   #### default: a=0.1, b=2
    if x > 0 :
        e = np.exp(1)
        ### constants calculations
        
        c1 = 1/(b**2*(a**2+a)*np.exp(a-1))
        c2 = 1 - np.exp(a)*(a*b+1)*c1
        c3 = -np.exp(a+1)*c1
        term = (x+e)*np.exp(a*np.log(x+e)**b) 
        return(c1*term + c2*x + c3)
    else:
        return(np.exp(x)-1)
    
def df_explog(x,a,b):
    if x > 0:
        e = np.exp(1)
        ### constants calculations
        
        c1 = 1/(b**2*(a**2+a)*np.exp(a-1))
        c2 = 1 - np.exp(a)*(a*b+1)*c1
        term = np.exp(a*(np.log(x+e))**b)*(a*b*(np.log(x+e))**(b-1)+1)
        return(c1*term + c2)
    else:
        return(np.exp(x))
    
    
def ddf_explog(x,a,b):
    if x > 0:
        e = np.exp(1)
        c1 = 1/(b**2*(a**2+a)*np.exp(a-1))
        
        term1 = a*b/(x+e)*np.exp(a*np.log(x+e)**b)
        term2 = (b-1)*np.log(x+e)**(b-2) + np.log(x+e)**(b-1) + a*b*np.log(x+e)**(2*b-2)
        return(c1*term1*term2)
    else:
        return(np.exp(x))

We now generate the log-normal samples $X_i$'s and define the parameters for the optimization problem:

In [126]:
np.random.seed(10)
X = -(np.random.lognormal(0,1,size= 20))
a = 0.1
b = 2
alpha = 0.05
r = 0.01

Next, we define the ipopt problem class where we specify the objective, gradient, constraint, jacobian and hessian functions. To do that, we first calculate the following partial derivatives of a perspective function, which are:
\begin{align*}
\frac{\partial}{\partial x}\left(\lambda \phi^*\left(\frac{x}{\lambda}\right)\right)&=(\phi^*)'\left(\frac{x}{\lambda}\right)\\
\frac{\partial}{\partial\lambda}\left(\lambda \phi^*\left(\frac{x}{\lambda}\right)\right)&=\phi^*\left(\frac{x}{\lambda}\right)-(\phi^*)'\left(\frac{x}{\lambda}\right)\cdot \frac{x}{\lambda}
\end{align*}

as well as the second partial derivatives:
\begin{align*}
\frac{\partial^2}{\partial x^2}\left(\lambda \phi^*\left(\frac{x}{\lambda}\right)\right)&=(\phi^*)''\left(\frac{x}{\lambda}\right)\cdot \frac{1}{\lambda}\\
\frac{\partial^2}{\partial\lambda^2}\left(\lambda \phi^*\left(\frac{x}{\lambda}\right)\right)&=(\phi^*)''\left(\frac{x}{\lambda}\right)\cdot \frac{x^2}{\lambda^3}\\
\frac{\partial^2}{\partial x\partial\lambda}\left(\lambda \phi^*\left(\frac{x}{\lambda}\right)\right)&=(\phi^*)''\left(\frac{x}{\lambda}\right)\cdot -\frac{x}{\lambda^2}.
\end{align*}

In [127]:
##### Ipopt with dummy variables


class MyOpti(cyipopt.Problem):     #### The above parameters are defined as global variables here, there is probably
                                        ####    a better way of doing this.
    
    def objective(self, x):
        """Returns the scalar value of the objective given x."""
        
        the1 = x[0]
        the2 = x[1]
        lbda = x[2]
        N = len(X)
        f_som = 0
        for i in range(3,N+3):
            f_som = f_som + lbda * f_explog((x[i]+the1)/lbda,a,b)
        return(f_som/N - the1 - the2 + lbda*r)

    def gradient(self, x):
        """Returns the gradient of the objective with respect to x."""
        
        the1 = x[0]
        the2 = x[1]
        lbda = x[2]
        N = len(X)
        grad = np.zeros(N+3)
        grad[1] = -1
        s1 = 0
        s2 = 0  
        for i in range(3,N+3):
            grad[i] = df_explog((x[i]+the1)/lbda,a,b)/N
            s1 = s1 + df_explog((x[i]+the1)/lbda,a,b)
            s2 = s2 + f_explog((x[i]+the1)/lbda,a,b) - df_explog((x[i]+the1)/lbda,a,b) * ((x[i]+the1)/lbda)
        grad[0] = -1 + s1/N
        grad[2] = r + s2/N
        return(grad)
             
        
       
    def constraints(self, x):
        """Returns the constraints."""
        
        N = len(X)
        cons = np.zeros(N)
        for i in range(N):
            cons[i] = 1/alpha * (x[1]-X[i])-x[i+3] 
        
        return(cons)

    
    def jacobian(self, x):
        """Returns the Jacobian of the constraints with respect to x."""
        
        N = len(X)
        jacob = np.zeros(N+3)
        jacob[1] = 1/alpha
        for i in range(3,N+3):
            jacob[i] = -1
        jacob = np.array([jacob])
        
        row, col = self.jacobianstructure()
        return(jacob[row,col])
        
        
    def jacobianstructure(self):
        N = len(X)
        jac_struc = np.zeros(N+3)+1
        jac_struc[0] = 0
        jac_struc[2] = 0
        jac_struc = np.array([jac_struc])
        return(np.nonzero(jac_struc))
    
    
    def hessianstructure(self):
        """Returns the row and column indices for non-zero vales of the
        Hessian."""

        # NOTE: The default hessian structure is of a lower triangular matrix,
        # therefore this function is redundant. It is included as an example
        # for structure callback.
       
        N = len(X)
        struc = np.tril(np.ones((N+3,N+3)))
        struc[1,:] = 0
        struc[0:N+3,1] = 0
        for i in range(3,N+1):
            struc[(i+1):(N+3),i] = 0
        struc[N+2,N+1] = 0
        return np.nonzero(struc)

    def hessian(self, x, lagrange, obj_factor):
        """Returns the non-zero values of the Hessian."""
        
        N = len(X)
        H = np.zeros((N+3,N+3))
        the1 = x[0]
        the2 = x[1]
        lbda = x[2]
        H00 = 0
        H20 = 0
        H22 = 0
        for i in range(3,N+3):
            H00 = H00 + ddf_explog((x[i]+the1)/lbda,a,b) * 1/lbda
            H20 = H20 - ddf_explog((x[i]+the1)/lbda,a,b) * (x[i]+the1)/lbda**2
            H22 = H22 + ddf_explog((x[i]+the1)/lbda,a,b) * ((x[i]+the1)**2)/lbda**3
            H[i,0] = ddf_explog((x[i]+the1)/lbda,a,b) * 1/lbda * 1/N
            H[i,2] = -ddf_explog((x[i]+the1)/lbda,a,b) * (x[i]+the1)/lbda**2 * 1/N
            H[i,i] = ddf_explog((x[i]+the1)/lbda,a,b) * 1/lbda * 1/N
        
        H[0,0] = H00/N
        H[2,0] = H20/N
        H[2,2] = H22/N        

        H = obj_factor*H

        row, col = self.hessianstructure()

        return H[row, col]


    def intermediate(self, alg_mod, iter_count, obj_value, inf_pr, inf_du, mu,
                     d_norm, regularization_size, alpha_du, alpha_pr,
                     ls_trials):
        """Prints information at every Ipopt iteration."""

        iterate = self.get_current_iterate()
        infeas = self.get_current_violations()
        primal = iterate["x"]
        jac = self.jacobian(primal)
        msg = "Objective value at iteration #{:d} is - {:g}"

        print(msg.format(iter_count, obj_value))
        print("Primal iterate:", primal[0:3])
        print('inf_pr:', inf_pr)
     

Below we define the bounds on the variables and the constraints:

In [136]:
lb = [-30,-30,0.01]
ub = [30, 30, 100]


cl = []
cu = []
x0 = [0.5,-2,1]
N = len(X)
for i in range(3, N+3):
    lb.append(0.01)
    ub.append(1000)
    cl.append(-100000)
    cu.append(0.01)
    x0.append(np.maximum(1/alpha*(x0[1]-X[i-3]),0)+0.001)

We define the problem for ipopt

In [137]:
nlp = MyOpti(
    n=len(x0),
    m=len(cl),
    lb=lb,
    ub=ub,
    cl=cl,
    cu=cu,
)
nlp.add_option('tol', 1e-6)
nlp.add_option('max_iter', 4000)

In [138]:
#### running ipopt

x, info = nlp.solve(x0)

Objective value at iteration #0 is - 106.388
Primal iterate: [ 0.5 -2.   1. ]
inf_pr: 0.0
Objective value at iteration #1 is - 14.4844
Primal iterate: [-3.72405971 -6.49660426  0.6212739 ]
inf_pr: 90.13028239611201
Objective value at iteration #2 is - 13.1334
Primal iterate: [-2.62034774 -1.18474468  0.79219166]
inf_pr: 82.35638030344893
Objective value at iteration #3 is - 7.85405
Primal iterate: [-7.57559684 -0.28978252  0.01782191]
inf_pr: 83.33370751680778
Objective value at iteration #4 is - 7.41713
Primal iterate: [-7.50308247  0.07468574  0.01921044]
inf_pr: 89.66281918611102
Objective value at iteration #5 is - 7.41324
Primal iterate: [-7.50352839  0.07944634  0.01932621]
inf_pr: 89.65267514418721
Objective value at iteration #6 is - 7.75652
Primal iterate: [-7.55622058  0.12879945  0.01983158]
inf_pr: 88.76820283255873
Objective value at iteration #7 is - 7.77886
Primal iterate: [-7.62512953  0.16063682  0.01948105]
inf_pr: 87.89579537277261
Objective value at iteration #8 is 

In [139]:
info

{'x': array([-26.21417216,  -2.27147116,   0.04805781,  30.3109987 ,
         30.75906908,  23.14730764,  22.70173969,  22.53311431,
         22.53311431,  22.53311431,  22.53311431,  22.53311431,
         22.53311431,  22.53311431,  10.70820575,  22.53311431,
         11.60445159,  22.53311431,  22.53311431,  22.53311431,
         22.53311431,  13.29886941,  22.53311431]),
 'g': array([ 5.13966116e-04, -3.52933515e+01, -6.43122007e+01, -4.82981389e+01,
        -3.07342738e+01, -5.82283252e+01, -4.18805782e+01, -4.56693575e+01,
        -4.78765244e+01, -5.11666836e+01, -3.71242054e+01,  1.04667049e+01,
        -6.03433738e+01, -1.10916305e+00, -4.28249960e+01, -3.67484384e+01,
        -6.15443865e+01, -4.50686683e+01,  2.95301428e+01, -6.11693016e+01]),
 'obj_val': 44.983468275286974,
 'mult_g': array([1.10214527e-05, 2.96008206e-09, 1.62417411e-09, 2.16293185e-09,
        3.39918864e-09, 1.79395304e-09, 2.49444937e-09, 2.28746568e-09,
        2.18198448e-09, 2.04163699e-09, 2.81408748